In [1]:
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv(r'D:\My Prog\Naresh_i _Technology\0_Task\NLP\Restaurant_Reviews.tsv', delimiter='\t')

In [3]:
df

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0


In [4]:
df = pd.concat([df]*2,ignore_index=True)
df

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
1995,I think food should have flavor and texture an...,0
1996,Appetite instantly gone.,0
1997,Overall I was not impressed and would not go b...,0
1998,"The whole experience was underwhelming, and I ...",0


In [5]:
df.shape

(2000, 2)

# Cleaning the dataset

In [6]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [7]:
cleaned_sent = []
for i in range(0,2000):
    review = re.sub(r'[^a-zA-Z]',' ',df['Review'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in set(stopwords.words('english'))]
    #review = [ps.stem(word) for word in review] # witout stopword
    review = ' '.join(review)
    cleaned_sent.append(review)

In [8]:
cleaned_sent

['wow love place',
 'crust good',
 'tasti textur nasti',
 'stop late may bank holiday rick steve recommend love',
 'select menu great price',
 'get angri want damn pho',
 'honeslti tast fresh',
 'potato like rubber could tell made ahead time kept warmer',
 'fri great',
 'great touch',
 'servic prompt',
 'would go back',
 'cashier care ever say still end wayyy overpr',
 'tri cape cod ravoli chicken cranberri mmmm',
 'disgust pretti sure human hair',
 'shock sign indic cash',
 'highli recommend',
 'waitress littl slow servic',
 'place worth time let alon vega',
 'like',
 'burritto blah',
 'food amaz',
 'servic also cute',
 'could care less interior beauti',
 'perform',
 'right red velvet cake ohhh stuff good',
 'never brought salad ask',
 'hole wall great mexican street taco friendli staff',
 'took hour get food tabl restaur food luke warm sever run around like total overwhelm',
 'worst salmon sashimi',
 'also combo like burger fri beer decent deal',
 'like final blow',
 'found place acc

In [9]:
len(cleaned_sent)

2000

# Creating bag of word model

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
x = cv.fit_transform(cleaned_sent).toarray()
print(x)
y = df.iloc[:,1].values
print(y)

# from sklearn.feature_extraction.text import TfidfVectorizer
# cv = TfidfVectorizer()
# X = cv.fit_transform(cleaned_sent).toarray()

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[1 0 0 ... 0 0 0]


# Splitting data into train and test set

In [11]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size = 0.2,random_state=0)


In [12]:
xtrain.shape

(1600, 1565)

In [13]:
ytrain.shape

(1600,)

In [14]:
xtest.shape

(400, 1565)

In [15]:
ytest.shape

(400,)

# Implementing naive bayes algorithm on this dataset

In [18]:
#Let's apply Logistic Regression
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression()
logit.fit(xtrain,ytrain)
# Prediction
logit_pred = logit.predict(xtest)
logit_pred
#Let's Calculate all other necessary information
# Accuracy
logit_ac = accuracy_score(ytest,logit_pred)
print('So the accuracy of the model is:',logit_ac)

# Bias
logit_bias = logit.score(xtrain,ytrain)
print('The bias of the model is:',logit_bias)
logit_variance = logit.score(xtest,ytest)
print('The variance of the model is:',logit_variance)
# Accuracy table
logit_cr = classification_report(ytest,logit_pred)
print('The classification report for logistic regression is:\n',logit_cr)
# table = logit.accuracy_score(ytest,logit_pred)



 # without cv = 0.8445
 # .8645 for 100 est





So the accuracy of the model is: 0.93
The bias of the model is: 0.975
The variance of the model is: 0.93
The classification report for logistic regression is:
               precision    recall  f1-score   support

           0       0.91      0.95      0.93       200
           1       0.95      0.91      0.93       200

    accuracy                           0.93       400
   macro avg       0.93      0.93      0.93       400
weighted avg       0.93      0.93      0.93       400



In [21]:
#Let's apply Naive Bayes using GridSearch
#First on Multinomial Naive Bayes

from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
# parameter grid
param_grid = {
    'alpha': [0.1, 0.5, 1.0],  # Smoothing parameter (Laplace smoothing)
    'fit_prior': [True, False]  # Whether to learn class prior probabilities
}
#Applying gridsearch cv
grid_search = GridSearchCV(estimator = mnb,param_grid = param_grid,cv = 5)
grid_search.fit(xtrain,ytrain)
best = grid_search.best_params_
best
mnb_pred = grid_search.predict(xtest)
mnb_pred
mnb_ac = accuracy_score(ytest,mnb_pred)
mnb_ac # without cv ac= 0.544
mnb_bias = grid_search.score(xtrain,ytrain)
mnb_bias # without cv 0.551875
mnb_variance = grid_search.score(xtest,ytest)
mnb_variance # without cv 0.544
mnb_cr = classification_report(ytest,mnb_pred)
mnb_cr

'              precision    recall  f1-score   support\n\n           0       0.88      0.92      0.90       200\n           1       0.92      0.88      0.90       200\n\n    accuracy                           0.90       400\n   macro avg       0.90      0.90      0.90       400\nweighted avg       0.90      0.90      0.90       400\n'

In [22]:
#Second on Gaussian Naive bayes

from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(xtrain,ytrain)
gnb_pred = gnb.predict(xtest)
gnb_pred
gnb_ac = accuracy_score(ytest,gnb_pred)
gnb_ac
gnb_bias = gnb.score(xtrain,ytrain)
gnb_bias
gnb_variance = gnb.score(xtest,ytest)
gnb_variance
gnb_cr = classification_report(ytest,gnb_pred)

In [23]:
#Let's go for Bernouli naive bayes algorithm

from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB()
bnb.fit(xtrain,ytrain)
bnb_pred = bnb.predict(xtest)
bnb_pred
bnb_ac = accuracy_score(ytest,bnb_pred)
bnb_ac
bnb_bias = bnb.score(xtrain,ytrain)
bnb_bias
bnb_variance = bnb.score(xtest,ytest)
bnb_variance

0.8775

In [24]:
#Let's apply Decision Tree using GridSearch
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
param_grid = {
    'criterion': ['gini', 'entropy'],  # Splitting criterion
    'splitter': ['best', 'random'],  # Strategy for choosing splits
    'max_depth': [None, 10, 20, 30, 40],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split a node
    'min_samples_leaf': [1, 2, 4],  # Minimum samples required at a leaf node
    'max_features': [None, 'sqrt', 'log2'],  # Number of features to consider for best split
}
model = GridSearchCV(estimator = dtc, param_grid = param_grid,cv = 5)
model.fit(xtrain,ytrain)
best_params = model.best_params_
best_params
'''
With cv the parameters are
{'criterion': 'entropy',
 'max_depth': 10,
 'max_features': None,
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'splitter': 'random'}
'''
dtc_pred = model.predict(xtest)
dtc_pred
dtc_ac = accuracy_score(ytest,dtc_pred)
dtc_ac # witout cv ac = 0.8445
dtc_bias = model.score(xtrain,ytrain)
dtc_bias # without cv = 0.882125
dtc_variance = model.score(xtest,ytest)
dtc_variance
dtc_cr = classification_report(ytest,dtc_pred)

In [32]:
#Let's apply Random Forest using GridSearchCv
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = 100)
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
#     'criterion': ['gini', 'entropy'],  # Splitting criterion for decision trees
#      'max_depth': [None, 10, 20, 30],  # Maximum depth of the trees
#     'min_samples_split': [2, 5, 10],  # Minimum samples required to split an internal node
#     'min_samples_leaf': [1, 2, 4],  # Minimum samples required to be at a leaf node
#     'max_features': ['auto', 'sqrt', 'log2'],  # Number of features to consider for the best split
#     'bootstrap': [True, False],  # Whether to bootstrap samples when building trees
#     'oob_score': [True, False],  # Whether to use out-of-bag samples for scoring
    'random_state': [0]  # Seed for random number generation
}
model = GridSearchCV(estimator=rfc,param_grid = param_grid, cv = 5)
model.fit(xtrain,ytrain)
best_params = model.best_params_
best_params
# Prediction
rfc_pred = model.predict(xtest)
rfc_pred
# Accuracy
rfc_ac = accuracy_score(ytest,rfc_pred)
rfc_ac # .8645 for 100 estimators , 
rfc_bias = model.score(xtrain,ytrain)
rfc_bias # 1.0 for 100 est
rfc_variance = model.score(xtest,ytest)
rfc_variance

0.95

In [26]:
#Let's Apply Support Vector Machine Algorithm
from sklearn.svm import SVC
svc_classifier = SVC()
# # Define the parameter grid
# param_grid = {
#     'C': [0.1],  # Regularization parameter
#     'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # Type of kernel
#     'degree': [2, 3, 4],  # Degree for 'poly' kernel
#     'gamma': ['scale', 'auto'],  # Kernel coefficient for 'rbf', 'poly', 'sigmoid'
#     'coef0': [0.0, 0.5, 1.0],  # Independent term in kernel function for 'poly', 'sigmoid'
#     'shrinking': [True, False],  # Use the shrinking heuristic
#     'probability': [True, False],  # Enable probability estimates
#     'tol': [1e-3, 1e-4, 1e-5],  # Tolerance for stopping criterion
#     'cache_size': [100, 200, 300],  # Size of the kernel cache (in MB)
#     'class_weight': [None, 'balanced'],  # Set class weights
#     'verbose': [True, False],  # Enable verbose output
#     'max_iter': [1000, 2000, -1],  # Hard limit on iterations within solver
#     'decision_function_shape': ['ovo', 'ovr'],  # Decision function shape for multi-class problems
#     'break_ties': [True, False],  # Whether to break ties according to confidence values
#     'random_state': [0, 42]  # Controls random number generation for probability estimates
# }
# model = GridSearchCV(estimator = svc_classifier, param_grid=param_grid)
svc_classifier.fit(xtrain,ytrain)
# Prediction
svc_pred = svc_classifier.predict(xtest)
svc_pred
# Accuracy
svc_ac = accuracy_score(ytest,svc_pred)
svc_ac
svc_bias = svc_classifier.score(xtrain,ytrain)
svc_bias
svc_variance = svc_classifier.score(xtest,ytest)
svc_variance
svc_cr = classification_report(ytest,svc_pred)
svc_cr


'              precision    recall  f1-score   support\n\n           0       0.93      0.93      0.93       200\n           1       0.93      0.93      0.93       200\n\n    accuracy                           0.93       400\n   macro avg       0.93      0.93      0.93       400\nweighted avg       0.93      0.93      0.93       400\n'

In [27]:
#Let's Apply KNN Algorithm
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
param_grid = {
    'n_neighbors': [3, 5, 7],  # Varying number of neighbors
    'weights': ['uniform', 'distance'],  # Types of weights
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # Different algorithms
    'leaf_size': [20, 30, 40],  # Varying leaf size
    'p': [1, 2],  # Power parameter for Minkowski metric
    'metric': ['euclidean', 'manhattan'],  # Distance metric options
}
model = GridSearchCV(estimator=knn, param_grid = param_grid)
model.fit(xtrain,ytrain)
best_params = model.best_params_
best_params
# Prediction
knn_pred = model.predict(xtest)
knn_pred
# Accuracy
knn_ac = accuracy_score(ytest,knn_pred)
knn_ac
# Bias
knn_bias = model.score(xtrain,ytrain)
knn_bias
knn_variance = model.score(xtest,ytest)
knn_variance
knn_cr = classification_report(ytest,knn_pred)
knn_cr

'              precision    recall  f1-score   support\n\n           0       0.90      0.94      0.92       200\n           1       0.94      0.90      0.92       200\n\n    accuracy                           0.92       400\n   macro avg       0.92      0.92      0.92       400\nweighted avg       0.92      0.92      0.92       400\n'

In [28]:
#Let's apply Gradient boosting algorithm
#XGBoost
from xgboost import XGBClassifier
xgb = XGBClassifier()
# Fit the model on the training data
xgb.fit(xtrain, ytrain)
# Make predictions on the test data
y_pred = xgb.predict(xtest)

# Evaluate the model
xgb_ac = accuracy_score(ytest, y_pred)
xgb_ac
xgb_bias = xgb.score(xtrain,ytrain)
xgb_bias
xgb_variance = xgb.score(xtest,ytest)
xgb_variance
print("Classification Report:")
xgb_cr = classification_report(ytest, y_pred)
# # Define the parameter grid for XGBoost classifier
# param_grid = {
#     'n_estimators': [100, 200],  # Number of boosting rounds
#     'max_depth': [3, 4, 5],  # Maximum depth of a tree
#     'learning_rate': [0.1, 0.01, 0.001],  # Learning rate
#     'subsample': [0.8, 0.9, 1.0],  # Subsample ratio of the training instances
#     'colsample_bytree': [0.8, 0.9, 1.0],  # Subsample ratio of columns when constructing each tree
#     'gamma': [0, 0.1, 0.2]  # Mihttp://localhost:8888/notebooks/My%20Prog/Naresh_i%20_Technology/Resume%20Project/Customer%20Churn%20Analysis/Untitled.ipynb#nimum loss reduction required to make a further partition
# }
# '''
# model = GridSearchCV(estimator=xgb, param_grid=param_grid, cv = 5)

Classification Report:


In [29]:
#Let's apply lightgbm
import lightgbm as lgb
lgb_classifier = lgb.LGBMClassifier()
param_grid = {
    "boosting_type": ["gbdt", "rf", "dart"],
    "num_leaves": [20, 31, 40],
    "max_depth": [5, 6, 7],
    "learning_rate": [0.05, 0.1],
    # Add more hyperparameters and their values
}
model = GridSearchCV(estimator = lgb_classifier, param_grid= param_grid,cv = 5)
model.fit(xtrain,ytrain)
best_params = model.best_params_
best_params
# Prediction
lgb_pred = model.predict(xtest)
lgb_pred
# Accuracy
lgb_ac = accuracy_score(ytest,lgb_pred)
lgb_ac
lgb_bias = model.score(xtrain,ytrain)
lgb_bias
lgb_variance = model.score(xtest,ytest)
lgb_variance
lgb_cr = classification_report(ytest,lgb_pred)
lgb_cr

[LightGBM] [Info] Number of positive: 640, number of negative: 640
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001428 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 141
[LightGBM] [Info] Number of data points in the train set: 1280, number of used features: 57
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Info] Number of positive: 640, number of negative: 640
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001265 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 128
[LightGBM] [Info] Number of data points in the train set: 1280, number of used features: 53
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive 

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 640, number of negative: 640
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000543 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 141
[LightGBM] [Info] Number of data points in the train set: 1280, number of used features: 57
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warnin

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 640, number of negative: 640
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000938 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 149
[LightGBM] [Info] Number of data points in the train set: 1280, number of used features: 61
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warnin

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Info] Number of positive: 640, number of negative: 640
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001497 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 138
[LightGBM] [Info] Number of data points in the train set: 1280, number of used features: 57
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Info] Number of positive: 800, number of negative: 800
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000828 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 184
[LightGBM] [Info] Number of data points in the train set: 1600, number of used features: 75
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive 

'              precision    recall  f1-score   support\n\n           0       0.65      0.87      0.75       200\n           1       0.81      0.54      0.65       200\n\n    accuracy                           0.70       400\n   macro avg       0.73      0.71      0.70       400\nweighted avg       0.73      0.70      0.70       400\n'

In [30]:
#Let's apply catboost
from catboost import CatBoostClassifier
cat_classifier = CatBoostClassifier()
cat_classifier.fit(xtrain,ytrain)
# catboost_param_grid = {
#     'iterations': [100, 200, 300],
#     'learning_rate': [0.1, 0.05, 0.01],
#     'depth': [4, 6, 8],
#     'l2_leaf_reg': [3.0, 5.0, 7.0],
#     'model_size_reg': [None, 0.1, 0.5],
#     'rsm': [0.7, 0.8, 1.0],
#     'loss_function': ['Logloss', 'CrossEntropy'],
#     'border_count': [254, 128],
#     # ... (additional parameters and their respective values)
#     'used_ram_limit': ['1.2gb', '2.4gb'],
#     'eval_fraction': [0.1, 0.2, 0.3],
#     'bagging_temperature': [0.5, 1.0, 1.5],
#     'bootstrap_type': ['Bayesian', 'Bernoulli', 'Poisson'],
#     # ... (more parameters and their respective values)
# }
# model = GridSearchCV(estimator = cat_classifier,param_grid = catboost_param_grid,cv = 5)
# Prediction
cat_pred = cat_classifier.predict(xtest)
cat_pred
# Accuracy
cat_ac = accuracy_score(ytest,cat_pred)
cat_ac
cat_bias = cat_classifier.score(xtrain,ytrain)
cat_bias
cat_variance = cat_classifier.score(xtest,ytest)
cat_variance
cat_cr = classification_report(ytest,cat_pred)
cat_cr

Learning rate set to 0.012592
0:	learn: 0.6898406	total: 194ms	remaining: 3m 13s
1:	learn: 0.6874081	total: 218ms	remaining: 1m 48s
2:	learn: 0.6843080	total: 241ms	remaining: 1m 20s
3:	learn: 0.6813697	total: 259ms	remaining: 1m 4s
4:	learn: 0.6781451	total: 274ms	remaining: 54.6s
5:	learn: 0.6757297	total: 290ms	remaining: 48.1s
6:	learn: 0.6731042	total: 306ms	remaining: 43.4s
7:	learn: 0.6710770	total: 322ms	remaining: 40s
8:	learn: 0.6686913	total: 339ms	remaining: 37.3s
9:	learn: 0.6658149	total: 358ms	remaining: 35.4s
10:	learn: 0.6632407	total: 377ms	remaining: 33.9s
11:	learn: 0.6610925	total: 400ms	remaining: 33s
12:	learn: 0.6596609	total: 416ms	remaining: 31.6s
13:	learn: 0.6572591	total: 435ms	remaining: 30.6s
14:	learn: 0.6552685	total: 452ms	remaining: 29.7s
15:	learn: 0.6536241	total: 467ms	remaining: 28.7s
16:	learn: 0.6519402	total: 485ms	remaining: 28s
17:	learn: 0.6499544	total: 503ms	remaining: 27.4s
18:	learn: 0.6481720	total: 519ms	remaining: 26.8s
19:	learn: 0.6

163:	learn: 0.5388274	total: 2.9s	remaining: 14.8s
164:	learn: 0.5383511	total: 2.92s	remaining: 14.8s
165:	learn: 0.5379138	total: 2.93s	remaining: 14.7s
166:	learn: 0.5375365	total: 2.94s	remaining: 14.7s
167:	learn: 0.5370955	total: 2.96s	remaining: 14.6s
168:	learn: 0.5366976	total: 2.97s	remaining: 14.6s
169:	learn: 0.5361594	total: 2.98s	remaining: 14.6s
170:	learn: 0.5357528	total: 3s	remaining: 14.5s
171:	learn: 0.5353413	total: 3.01s	remaining: 14.5s
172:	learn: 0.5351333	total: 3.02s	remaining: 14.5s
173:	learn: 0.5347950	total: 3.04s	remaining: 14.4s
174:	learn: 0.5344215	total: 3.05s	remaining: 14.4s
175:	learn: 0.5338116	total: 3.06s	remaining: 14.3s
176:	learn: 0.5333801	total: 3.08s	remaining: 14.3s
177:	learn: 0.5329222	total: 3.09s	remaining: 14.3s
178:	learn: 0.5324472	total: 3.1s	remaining: 14.2s
179:	learn: 0.5321374	total: 3.12s	remaining: 14.2s
180:	learn: 0.5315962	total: 3.13s	remaining: 14.2s
181:	learn: 0.5311681	total: 3.15s	remaining: 14.1s
182:	learn: 0.530

325:	learn: 0.4848544	total: 6.67s	remaining: 13.8s
326:	learn: 0.4845314	total: 6.69s	remaining: 13.8s
327:	learn: 0.4843153	total: 6.71s	remaining: 13.8s
328:	learn: 0.4841005	total: 6.74s	remaining: 13.7s
329:	learn: 0.4839529	total: 6.75s	remaining: 13.7s
330:	learn: 0.4836293	total: 6.78s	remaining: 13.7s
331:	learn: 0.4832580	total: 6.79s	remaining: 13.7s
332:	learn: 0.4830246	total: 6.82s	remaining: 13.7s
333:	learn: 0.4828561	total: 6.84s	remaining: 13.6s
334:	learn: 0.4826618	total: 6.86s	remaining: 13.6s
335:	learn: 0.4822927	total: 6.88s	remaining: 13.6s
336:	learn: 0.4820699	total: 6.91s	remaining: 13.6s
337:	learn: 0.4814866	total: 6.92s	remaining: 13.6s
338:	learn: 0.4810662	total: 6.94s	remaining: 13.5s
339:	learn: 0.4807948	total: 6.96s	remaining: 13.5s
340:	learn: 0.4805564	total: 6.98s	remaining: 13.5s
341:	learn: 0.4802357	total: 7s	remaining: 13.5s
342:	learn: 0.4799120	total: 7.02s	remaining: 13.4s
343:	learn: 0.4796931	total: 7.04s	remaining: 13.4s
344:	learn: 0.4

488:	learn: 0.4462209	total: 10.4s	remaining: 10.9s
489:	learn: 0.4458582	total: 10.4s	remaining: 10.8s
490:	learn: 0.4456469	total: 10.4s	remaining: 10.8s
491:	learn: 0.4454318	total: 10.4s	remaining: 10.8s
492:	learn: 0.4452654	total: 10.5s	remaining: 10.8s
493:	learn: 0.4449731	total: 10.5s	remaining: 10.7s
494:	learn: 0.4447234	total: 10.5s	remaining: 10.7s
495:	learn: 0.4444607	total: 10.5s	remaining: 10.7s
496:	learn: 0.4442215	total: 10.5s	remaining: 10.7s
497:	learn: 0.4439255	total: 10.6s	remaining: 10.6s
498:	learn: 0.4436579	total: 10.6s	remaining: 10.6s
499:	learn: 0.4434750	total: 10.6s	remaining: 10.6s
500:	learn: 0.4431789	total: 10.6s	remaining: 10.6s
501:	learn: 0.4430354	total: 10.6s	remaining: 10.5s
502:	learn: 0.4427840	total: 10.7s	remaining: 10.5s
503:	learn: 0.4426670	total: 10.7s	remaining: 10.5s
504:	learn: 0.4424823	total: 10.7s	remaining: 10.5s
505:	learn: 0.4423177	total: 10.7s	remaining: 10.5s
506:	learn: 0.4420755	total: 10.7s	remaining: 10.4s
507:	learn: 

652:	learn: 0.4075777	total: 13.3s	remaining: 7.08s
653:	learn: 0.4073695	total: 13.3s	remaining: 7.06s
654:	learn: 0.4070856	total: 13.4s	remaining: 7.04s
655:	learn: 0.4068862	total: 13.4s	remaining: 7.02s
656:	learn: 0.4066463	total: 13.4s	remaining: 7s
657:	learn: 0.4063395	total: 13.4s	remaining: 6.98s
658:	learn: 0.4060484	total: 13.4s	remaining: 6.96s
659:	learn: 0.4058444	total: 13.5s	remaining: 6.93s
660:	learn: 0.4056536	total: 13.5s	remaining: 6.91s
661:	learn: 0.4054487	total: 13.5s	remaining: 6.89s
662:	learn: 0.4052931	total: 13.5s	remaining: 6.87s
663:	learn: 0.4049724	total: 13.5s	remaining: 6.85s
664:	learn: 0.4048036	total: 13.6s	remaining: 6.83s
665:	learn: 0.4045904	total: 13.6s	remaining: 6.8s
666:	learn: 0.4043032	total: 13.6s	remaining: 6.78s
667:	learn: 0.4040657	total: 13.6s	remaining: 6.76s
668:	learn: 0.4038641	total: 13.6s	remaining: 6.74s
669:	learn: 0.4034584	total: 13.6s	remaining: 6.71s
670:	learn: 0.4032202	total: 13.6s	remaining: 6.69s
671:	learn: 0.40

820:	learn: 0.3690619	total: 16.1s	remaining: 3.5s
821:	learn: 0.3689626	total: 16.1s	remaining: 3.48s
822:	learn: 0.3688212	total: 16.1s	remaining: 3.46s
823:	learn: 0.3685235	total: 16.1s	remaining: 3.44s
824:	learn: 0.3684182	total: 16.1s	remaining: 3.42s
825:	learn: 0.3681006	total: 16.1s	remaining: 3.4s
826:	learn: 0.3678584	total: 16.1s	remaining: 3.38s
827:	learn: 0.3676032	total: 16.2s	remaining: 3.36s
828:	learn: 0.3674614	total: 16.2s	remaining: 3.33s
829:	learn: 0.3672730	total: 16.2s	remaining: 3.31s
830:	learn: 0.3669578	total: 16.2s	remaining: 3.29s
831:	learn: 0.3666423	total: 16.2s	remaining: 3.27s
832:	learn: 0.3663932	total: 16.2s	remaining: 3.25s
833:	learn: 0.3663083	total: 16.2s	remaining: 3.23s
834:	learn: 0.3661879	total: 16.3s	remaining: 3.21s
835:	learn: 0.3659008	total: 16.3s	remaining: 3.19s
836:	learn: 0.3657356	total: 16.3s	remaining: 3.17s
837:	learn: 0.3656153	total: 16.3s	remaining: 3.15s
838:	learn: 0.3652971	total: 16.3s	remaining: 3.13s
839:	learn: 0.

982:	learn: 0.3387165	total: 18.4s	remaining: 317ms
983:	learn: 0.3385299	total: 18.4s	remaining: 299ms
984:	learn: 0.3383943	total: 18.4s	remaining: 280ms
985:	learn: 0.3380179	total: 18.4s	remaining: 261ms
986:	learn: 0.3379140	total: 18.4s	remaining: 243ms
987:	learn: 0.3377415	total: 18.4s	remaining: 224ms
988:	learn: 0.3375258	total: 18.4s	remaining: 205ms
989:	learn: 0.3372873	total: 18.5s	remaining: 186ms
990:	learn: 0.3370601	total: 18.5s	remaining: 168ms
991:	learn: 0.3369189	total: 18.5s	remaining: 149ms
992:	learn: 0.3365648	total: 18.5s	remaining: 130ms
993:	learn: 0.3362836	total: 18.5s	remaining: 112ms
994:	learn: 0.3362024	total: 18.5s	remaining: 93.1ms
995:	learn: 0.3359645	total: 18.5s	remaining: 74.4ms
996:	learn: 0.3356208	total: 18.5s	remaining: 55.8ms
997:	learn: 0.3355266	total: 18.6s	remaining: 37.2ms
998:	learn: 0.3353282	total: 18.6s	remaining: 18.6ms
999:	learn: 0.3351683	total: 18.6s	remaining: 0us


'              precision    recall  f1-score   support\n\n           0       0.80      0.93      0.86       200\n           1       0.91      0.78      0.84       200\n\n    accuracy                           0.85       400\n   macro avg       0.86      0.85      0.85       400\nweighted avg       0.86      0.85      0.85       400\n'

In [37]:
algo = ['Logistic Regression','Bernouli Naive Bayes','Gaussian Naive Bayes','MultiNomial Naive Bayes','Decision Trees','Random Forest',
        'Support Vector Machines (SVM)','k-Nearest Neighbors (k-NN)','XGBoost', 'LightGBM', 'CatBoost']
accuracy = [logit_ac,bnb_ac,gnb_ac,mnb_ac,dtc_ac,rfc_ac,svc_ac,knn_ac,xgb_ac,lgb_ac,cat_ac]
bias = [logit_bias,bnb_bias,gnb_bias,mnb_bias,dtc_bias,rfc_bias,svc_bias,knn_bias,xgb_bias,lgb_bias,cat_bias]
variance = [logit_variance,bnb_variance,gnb_variance,mnb_variance,dtc_variance,rfc_variance,svc_variance,knn_variance,xgb_variance,lgb_variance,cat_variance]
# print('Including Stopwords\n')
for model, acc, bias, variance in zip(algo, accuracy,bias,variance):
    print(f"{model}Model's  Accuracy - {acc}")
    print(f"Model's Bias:-{bias} & Variance:- {variance}\n")

Logistic RegressionModel's  Accuracy - 0.93
Model's Bias:-0.975 & Variance:- 0.93

Bernouli Naive BayesModel's  Accuracy - 0.8775
Model's Bias:-0.935625 & Variance:- 0.8775

Gaussian Naive BayesModel's  Accuracy - 0.8575
Model's Bias:-0.915625 & Variance:- 0.8575

MultiNomial Naive BayesModel's  Accuracy - 0.9
Model's Bias:-0.955 & Variance:- 0.9

Decision TreesModel's  Accuracy - 0.93
Model's Bias:-0.9975 & Variance:- 0.93

Random ForestModel's  Accuracy - 0.95
Model's Bias:-0.9975 & Variance:- 0.95

Support Vector Machines (SVM)Model's  Accuracy - 0.93
Model's Bias:-0.9825 & Variance:- 0.93

k-Nearest Neighbors (k-NN)Model's  Accuracy - 0.92
Model's Bias:-0.9975 & Variance:- 0.92

XGBoostModel's  Accuracy - 0.8425
Model's Bias:-0.900625 & Variance:- 0.8425

LightGBMModel's  Accuracy - 0.705
Model's Bias:-0.76875 & Variance:- 0.705

CatBoostModel's  Accuracy - 0.85
Model's Bias:-0.91625 & Variance:- 0.85

